In [18]:
# Imports
import pyxdf
import mne
from mne.datasets import misc
import os
import time
from pyxdf import match_streaminfos, resolve_streams
from mnelab.io.xdf import read_raw_xdf
import json
import csv
import pybv
from bids_validator import BIDSValidator

## Check for existence of a new xdf file in the directory structure

In [2]:
def check_for_new_file(folder_path, last_checked_file_path):
    """
    Checks for new files in a multilevel folder structure.
    
    Parameters:
        root_dir (str): The root directory to search for new files.
        last_checked_time (float): The last checked time in seconds since the epoch.
    
    Returns:
        list: A list of new file paths.
    """
    # Retrieve the last checked time from the file
    try:
        with open(last_checked_file_path, 'r') as f:
            last_checked_time = float(f.readlines()[-1].strip())

    except FileNotFoundError:
        last_checked_time = 0.0
    

    # Check for new files
    new_files = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            if os.path.getmtime(file_path) > last_checked_time:
                new_files.append(file_path)
    
    # Save the current time to the file
    with open(last_checked_file_path, 'a') as f:
        f.write(str(time.time()) + '\n')
        
    
    return new_files

## Load the xdf file from the given path and get its streams

In [3]:
def get_the_streams(xdf_path):
    streams = resolve_streams(xdf_path)
    stream_names = [streams[i]['name'] for i in range(len(streams))]
    return stream_names,streams

In [4]:
def create_raw_xdf(xdf_path,streams):
    # Get the stream id of the EEG stream
    stream_id = match_streaminfos(streams, [{"type": "EEG"}])[0]
    raw = read_raw_xdf(xdf_path,stream_ids=[stream_id])
    return raw


### Create the Metadata files

In [5]:
def create_participants_json(file_path):
    data = {
        "pInfoDesc": {
            "participant_id": {
                "Description": "Unique participant identifier"
            },
            "gender": {
                "Description": "Sex of participant",
                "Levels": {
                    "M": "Male",
                    "F": "Female"
                }
            },
            "age": {
                "Description": "Age of participant",
                "Units": "Years"
            },
            "handedness": {
                "Description": "Handedness of participant",
                "Levels": {
                    "R": "Right",
                    "L": "Left"
                }
            },
            "vision": {
                "Description": "Vision of participant",
                "Levels": {
                    "N": "Normal",
                    "C": "Corrected"
                }
            }
        }
    }
    file = file_path + '/participants.json'
    with open(file, "w") as json_file:
        json.dump(data, json_file, indent=4)

    return file


In [6]:
def create_eeg_json(file_path):
    tInfo = {
        "InstitutionName": "University of Stuttgart",
        "InstitutionAddress": "Universitätsstraße 34, 70569 Stuttgart, GER",
        "InstitutionalDepartmentName": "Computational Cognitive Science",
        "Manufacturer": "ANT Neuro",
        "ManufacturersModelName": "EEG Cap: waveguard original cap (CA-203.s1). Amplifier: eego sports (EE-225).",
        "TaskDescription": "",
        "Instructions": "",
        "EEGReference": "CPz",
        "PowerLineFrequency": 50,
        "SoftwareFilter": "n/a",
        "RecordingType": "continuous",
        "EEGGround": "left earlobe",
        "EEGPlacementScheme": "10-5"
    }
    '''
    if switch_task == 1:
        tInfo["TaskDescription"] = "The P300 component was elicited in an active visual oddball task, adapted from the ERP Core (Kappenman et al., 2021). The letters A, B, C, D, and E were presented in random order (p = .2 for each letter). For each block one letter was designated the target and the other 4 letters were distracters. Each letter was a target in 2 blocks and a distractor in the other 8 blocks. Participants responded whether the presented letter was the target or a distracter on each trial. Target-response mappings were randomized."
        tInfo["Instructions"] = "Throughout this experiment you will see a stream of letters (ABCDE). Your task is to respond to the letter that was displayed by pressing either the [TARGET_BUTTON] or [DISTRACTER_BUTTON], depending on the assignment given at the beginning of each block. You can take pauses between blocks if required. Press the buttons with your left and right index fingers. Maintain fixation on the cross in the screen center. Respond as quickly and accurately as possible."
    elif switch_task == 2:
        tInfo["TaskDescription"] = "The N170 component was elicited in an active visual distracter task. Images of neutral faces, taken from the Chicago Face Database (Ma et al., 2015) were presented in random order. Subjects fixated a fixation cross in the screen center and responded when a red dot (=distracter) flickered in the center of the fixation cross. Distracters (p = .1 for each image) were spaced by at least 4 s."
        tInfo["Instructions"] = "During this experiment faces will be presented, either without or with small breaks in between. Your task is to fixate the cross in the screen center and press the [BUTTON], as soon as you see a red dot flickering in the center of the cross. You can take pauses between blocks if required. Press the [BUTTON] with your right index finger. Respond as quickly as possible."
    '''
    file = file_path + '/eeg.json'
    with open(file, "w") as json_file:
        json.dump(tInfo, json_file, indent=4)
        return file

In [7]:
def create_events_json(file_path):

    eInfo = {
        'onset': 'latency',
        'duration': 'duration',
        'sample': 'latency',
        'trial_type': 'type',
        'response_time': 'response_time',
        'keycode': 'keycode',
        'target_response': 'target_response'
    }
    
    eInfoDesc = {
        'keycode': {
            'Description': 'Keycode for response',
            'Levels': {
                'left': '11',
                'right': '12'
            }
        },
        'target_response': {
            'Description': 'Target-response mapping',
            'Levels': {
                'left': 'Left response (=keycode: 11) is correct for target trials with target_response=left',
                'right': 'Right response (=keycode: 12) is correct for target trials with target_response=right'
            }
        }
    }
    
    data = {
        'eInfo': eInfo,
        'eInfoDesc': eInfoDesc
    }
    file = file_path + '/events.json'
    
    with open(file, 'w') as json_file:
        json.dump(data, json_file, indent=4)
        return file

In [8]:
def create_dataset_description_json(file_path):
    general_info = {
        'Name': 'P300 visual oddball task + N170 visual distracter task',
        'ReferencesAndLinks': ['No bibliographic reference other than the DOI for this dataset'],
        'BIDSVersion': 'v1.7.0',
        'License': 'CC-BY',
        'Authors': ['Martin Geiger']
    }

    data = {
        'general_info': general_info
    }

    file = file_path + '/dataset_description.json'
    with open(file, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    return file

In [9]:
def create_participants_tsv(file_path):
    pInfo = [
        ['gender', 'age', 'handedness', 'vision'],
        ['M', 25, 'R', 'N'],
        ['M', 26, 'R', 'N'],
        ['M', 29, 'R', 'C'],
        ['M', 27, 'R', 'C'],
        ['F', 29, 'R', 'N'],
        ['M', 36, 'R', 'N'],
        ['M', 28, 'R', 'C'],
        ['M', 30, 'R', 'N'],
        ['M', 27, 'R', 'N'],
        ['M', 23, 'R', 'N'],
        ['F', 26, 'R', 'C'],
        ['F', 28, 'R', 'C'],
        ['M', 30, 'R', 'N'],
        ['M', 34, 'R', 'C'],
        ['M', 24, 'R', 'C'],
        ['M', 29, 'R', 'N'],
        ['F', 26, 'R', 'C'],
        ['M', 30, 'R', 'N'],
        ['F', 30, 'R', 'N'],
        ['M', 28, 'R', 'N'],
        ['M', 26, 'R', 'N'],
        ['F', 21, 'R', 'N'],
        ['M', 32, 'R', 'N'],
        ['M', 26, 'R', 'C'],
        ['M', 30, 'R', 'C'],
        ['M', 28, 'R', 'N'],
        ['M', 27, 'R', 'N'],
        ['W', 29, 'R', 'N'],
        ['M', 23, 'R', 'C'],
        ['F', 25, 'R', 'C'],
        ['M', 30, 'R', 'N'],
        ['F', 27, 'R', 'N'],
        ['M', 33, 'R', 'C'],
        ['F', 24, 'R', 'N'],
        ['F', 25, 'R', 'N'],
        ['M', 27, 'R', 'N'],
        ['F', 22, 'R', 'N'],
        ['F', 25, 'R', 'N'],
        ['M', 22, 'R', 'N'],
        ['F', 23, 'R', 'N'],
        ['M', 23, 'R', 'N']
    ]
    file = file_path + '/participants.tsv'
    with open(file, 'w', newline='') as tsv_file:
        writer = csv.writer(tsv_file, delimiter='\t')
        writer.writerows(pInfo)
    return file

In [10]:
def gen_file_name(output_dir,file_prefix,ext):
     return os.path.join(output_dir, file_prefix + ext)

In [11]:
def create_bids_files(xdf_file, output_dir, participants_file, events_file, dataset_desc_file, eeg_desc_file, participants_desc_file):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Generate EEG file paths
    xdf_file_prefix = os.path.splitext(os.path.basename(xdf_file))[0]
    vhdr_file = gen_file_name(output_bids,xdf_file_prefix,'.vhdr')
    vmrk_file = gen_file_name(output_bids,xdf_file_prefix,'.vmrk')
    eeg_file = gen_file_name(output_bids,xdf_file_prefix,'.eeg')
    # Create .vhdr file
    with open(vhdr_file, 'w') as file:
        file.write(f'Brain Vision Data Exchange Header File Version 1.0\n')


    # Create .vmrk file
    with open(vmrk_file, 'w') as file:
        file.write(f'Brain Vision Data Exchange Marker File Version 1.0\n')
       

    # Create .eeg file (empty file)
    open(eeg_file, 'a').close()

    # Copy participants.tsv file
    participants_output_file = os.path.join(output_dir, 'participants.tsv')
    os.replace(participants_file, participants_output_file)

    # Copy events.json file
    events_output_file = os.path.join(output_dir, 'events.json')
    os.replace(events_file, events_output_file)

    # Copy dataset_description.json file
    dataset_desc_output_file = os.path.join(output_dir, 'dataset_description.json')
    os.replace(dataset_desc_file, dataset_desc_output_file)

    # Copy eeg.json file
    eeg_desc_output_file = os.path.join(output_dir, 'eeg.json')
    os.replace(eeg_desc_file, eeg_desc_output_file)

    # Copy participants.json file
    participants_desc_output_file = os.path.join(output_dir, 'participants.json')
    os.replace(participants_desc_file, participants_desc_output_file)

    return vhdr_file, vmrk_file, eeg_file


In [25]:
def validate_bids_files(bids_dir):
    # write a validator function
    return 0

#### Main Function

In [13]:
# Check for new files
# The path is in the sub-xxx/ses-xxx structure.
PATH = 'xdf_files/'
last_checked_file_path = 'last_time_checked.txt'
check_for_new_file(PATH, last_checked_file_path)

[]

In [14]:
xdf_file = 'sample_data/raw_xdf/sub-004/ses-001/eeg/sub-004_ses-001_task-Duration_run-001_eeg.xdf'
stream_names,streams = get_the_streams(xdf_file)
raw = create_raw_xdf(xdf_file,streams)

Creating RawArray with float64 data, n_channels=65, n_times=923008
    Range : 0 ... 923007 =      0.000 ...   922.999 secs
Ready.


In [15]:
# Populate the metadata files
file_path = 'metadata_bids' # add file path for main function
part_desc_file = create_participants_json(file_path)

eeg_desc_file = create_eeg_json(file_path)

events_file = create_events_json(file_path)

dataset_desc_file = create_dataset_description_json(file_path)

participants_file = create_participants_tsv(file_path)

In [16]:
# create BIDS directory
output_bids = 'test_bids_dir'
vhdr_file, vmrk_file, eeg_file = create_bids_files(xdf_file,output_bids, participants_file, events_file, dataset_desc_file, eeg_desc_file, part_desc_file)

print(f'Created BIDS files: {vhdr_file}, {vmrk_file}, {eeg_file}')

Created BIDS files: test_bids_dir/sub-004_ses-001_task-Duration_run-001_eeg.vhdr, test_bids_dir/sub-004_ses-001_task-Duration_run-001_eeg.vmrk, test_bids_dir/sub-004_ses-001_task-Duration_run-001_eeg.eeg


In [26]:
validate_bids_files(output_bids)

0

##### TODO 
 https://github.com/cbrnr/bci_event_2021
 .bidsignore